<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/%E7%8C%AB%E7%8B%97%E6%95%B0%E6%8D%AE%E9%9B%86%E8%87%AA%E5%AE%9A%E4%B9%89%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/dataset"
os.chdir(path)
os.listdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['2_class', 'dc_2000']

In [0]:
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import glob#提取图像的路径

In [0]:
train_image_path=glob.glob('/content/drive/My Drive/dataset/dc_2000/train/*//*.jpg')

In [4]:
len(train_image_path)

2000

In [5]:
train_image_path[:5]

['/content/drive/My Drive/dataset/dc_2000/train/cat/cat.0.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.1.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.10.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.100.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/cat/cat.101.jpg']

In [6]:
train_image_path[-5:]

['/content/drive/My Drive/dataset/dc_2000/train/dog/dog.997.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.998.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.999.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.992.jpg',
 '/content/drive/My Drive/dataset/dc_2000/train/dog/dog.993.jpg']

##进行路径拆分，到他到底是猫还是狗

In [7]:
p='/content/drive/My Drive/dataset/dc_2000/train/dog/dog.993.jpg'
p.split('/')[7]

'dog'

In [0]:
#我们这里就可以实现把列表的dog和cat变成0和1这样就实现了trainlabel
#这样cat就会变成1 因为是转换成了int型
train_image_label=[int(p.split('/')[7]=='cat') for p in train_image_path]

In [9]:
train_image_label[-5:]

[0, 0, 0, 0, 0]

##定义函数读取路径

In [0]:
def load_preprosess_image(path,label):
  image=tf.io.read_file(path)#读取这个路径
  image=tf.image.decode_jpeg(image,channels=3)#解码数据
  #把图像划分到同样大小
  image=tf.image.resize(image,[256,256])
  #改变数据类型把uint8变成float32
  image=tf.cast(image,tf.float32)
  #归一化
  image=image/255
  #处理标签,把连着的label断开
  #[1,2,3]→[1][2][3]这个样子
  label=tf.reshape(label,[1])
  return image,label

##创建数据集

In [0]:
#创建数据集
train_image_ds=tf.data.Dataset.from_tensor_slices((train_image_path,train_image_label))
AUTOTUNE=tf.data.experimental.AUTOTUNE
#实现针对于cpu智能的并行的创建数据集
train_image_ds=train_image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)

In [12]:
train_image_ds

<ParallelMapDataset shapes: ((256, 256, 3), (1,)), types: (tf.float32, tf.int32)>

In [0]:
#设置batchsize
BATCH_SIZE=128#根据性能测试
train_count=len(train_image_path)
train_image_ds=train_image_ds.shuffle(train_count).batch(BATCH_SIZE)
#为了加速训练过程
train_image_ds=train_image_ds.prefetch(AUTOTUNE)

In [0]:
#这里的化他是一个eage模式的 他是可以迭代的，我们尝试着从中取出一个看一下
image,labels=next(iter(train_image_ds))

In [15]:
image.shape

TensorShape([128, 256, 256, 3])

In [16]:
labels.shape

TensorShape([128, 1])

##创建模型

In [0]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(1024,(3,3),activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(1)
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 512)       1

In [22]:
pred=model(image)

<tf.Tensor: shape=(128, 1), dtype=float32, numpy=
array([[-3.53030395e-04],
       [-1.25817023e-04],
       [ 1.06327701e-04],
       [ 5.09545207e-05],
       [-2.35504843e-03],
       [-2.89998483e-04],
       [-9.64557752e-04],
       [-4.26722877e-03],
       [-9.66471620e-04],
       [-2.64123548e-04],
       [-3.07588070e-03],
       [-1.17429066e-03],
       [-1.44978124e-03],
       [-1.18717970e-03],
       [-1.41829299e-03],
       [-9.57414508e-04],
       [-1.96587807e-03],
       [-1.63169648e-03],
       [-1.48638873e-03],
       [-3.09932046e-03],
       [ 5.03637828e-04],
       [-2.14022538e-03],
       [-3.68643785e-03],
       [-2.93535716e-03],
       [-2.46106554e-03],
       [-2.31385138e-03],
       [-1.91855198e-03],
       [ 1.88092003e-03],
       [-3.01395636e-03],
       [-3.27314553e-03],
       [-3.46944435e-04],
       [-6.46645669e-04],
       [-3.61922500e-03],
       [-1.27682881e-03],
       [-9.82807716e-04],
       [-1.09906239e-03],
       [-1.349

In [0]:
tf.cast(pred>0,tf.int32)